# PySpark CSV

* [CSV Files](https://spark.apache.org/docs/latest/sql-data-sources-csv.html)

> Spark SQL provides spark.read().csv("file_name") to read a file or directory of files in CSV format into Spark DataFrame, and dataframe.write().csv("path") to write to a CSV file. Function option() can be used to customize the behavior of reading or writing, such as controlling behavior of the header, delimiter character, character set, and so on.

* [PySpark Read CSV file into DataFrame](https://sparkbyexamples.com/pyspark/pyspark-read-csv-file-into-dataframe/)

In [1]:
%%html
<style>
table {float:left}
</style>

In [2]:
%%html
<style>
div.output_area pre {
    white-space: pre;
}
</style>

In [3]:
import os
import sys
import gc
from datetime import (
    datetime,
    date
)

#  Environemnt Variables

## Hadoop

In [4]:
os.environ['HADOOP_CONF_DIR'] = "/opt/hadoop/hadoop-3.2.2/etc/hadoop"

In [5]:
%%bash
export HADOOP_CONF_DIR="/opt/hadoop/hadoop-3.2.2/etc/hadoop"
ls $HADOOP_CONF_DIR | head -n 5

capacity-scheduler.xml
configuration.xsl
container-executor.cfg
core-site.xml
core-site.xml.48132.2022-02-15@12:29:41~


## PYTHONPATH

Refer to the **pyspark** modules to load from the ```$SPARK_HOME/python/lib``` in the Spark installation.

* [PySpark Getting Started](https://spark.apache.org/docs/latest/api/python/getting_started/install.html)

> Ensure the SPARK_HOME environment variable points to the directory where the tar file has been extracted. Update PYTHONPATH environment variable such that it can find the PySpark and Py4J under SPARK_HOME/python/lib. One example of doing this is shown below:

```
export PYTHONPATH=$(ZIPS=("$SPARK_HOME"/python/lib/*.zip); IFS=:; echo "${ZIPS[*]}"):$PYTHONPATH
```

Alternatively install **pyspark** with pip or conda locally which installs the Spark runtime libararies (for standalone).

* [Can PySpark work without Spark?](https://stackoverflow.com/questions/51728177/can-pyspark-work-without-spark)

> As of v2.2, executing pip install pyspark will install Spark. If you're going to use Pyspark it's clearly the simplest way to get started. On my system Spark is installed inside my virtual environment (miniconda) at lib/python3.6/site-packages/pyspark/jars  
> PySpark has a Spark installation installed. If installed through pip3, you can find it with pip3 show pyspark. Ex. for me it is at ~/.local/lib/python3.8/site-packages/pyspark. This is a standalone configuration so it can't be used for managing clusters like a full Spark installation.

In [6]:
# os.environ['PYTHONPATH'] = "/opt/spark/spark-3.1.2/python/lib/py4j-0.10.9-src.zip:/opt/spark/spark-3.1.2/python/lib/pyspark.zip"
sys.path.extend([
    "/opt/spark/spark-3.1.2/python/lib/py4j-0.10.9-src.zip",
    "/opt/spark/spark-3.1.2/python/lib/pyspark.zip"
])

## Python packages

Execute after the PYTHONPATH setup.

### pyspark.sql.funtions

See [pyspark.sql.functions module](https://spark.apache.org/docs/2.4.0/api/python/pyspark.sql.html#module-pyspark.sql.functions) for available function you can import. Spark Documentation [Built-in Functions](https://spark.apache.org/docs/latest/api/sql/index.html#day) has functions such as ```day```, ```month``` but they cannot be imported and [pyspark.sql.functions module](https://spark.apache.org/docs/2.4.0/api/python/pyspark.sql.html#module-pyspark.sql.functions) does not have them.

In [7]:
import pyspark.sql 
from pyspark.sql.types import *
from pyspark.sql.functions import (
    col,
    lit,
    avg,
    stddev,
    isnan,
    to_date,
    to_timestamp,
    date_format,
    year,
    month,
    hour,
    min,
    second,
)

---
# Spark Session


In [8]:
from pyspark.sql import SparkSession

In [9]:
spark = SparkSession.builder\
    .master('yarn') \
    .config('spark.submit.deployMode', 'client') \
    .config('spark.debug.maxToStringFields', 100) \
    .config('spark.executor.memory', '2g') \
    .getOrCreate()

2022-02-20 19:40:53,883 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2022-02-20 19:40:56,300 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
2022-02-20 19:40:58,938 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [10]:
NUM_CORES = 4
NUM_PARTITIONS = 3

spark.conf.set("spark.sql.shuffle.partitions", NUM_CORES * NUM_PARTITIONS)
spark.conf.set("spark.default.parallelism", NUM_CORES * NUM_PARTITIONS)
spark.conf.set('spark.sql.legacy.timeParserPolicy', 'LEGACY')

# Date/Time Format String

* [Datetime Patterns for Formatting and Parsing](https://spark.apache.org/docs/latest/sql-ref-datetime-pattern.html)


| Symbol | Meaning                      | Presentation | Examples                                       |
|--------|------------------------------|--------------|------------------------------------------------|
| G      | era                          | text         | AD; Anno Domini                                |
| y      | year                         | year         | 2020; 20                                       |
| D      | day-of-year                  | number(3)    | 189                                            |
| M/L    | month-of-year                | month        | 7; 07; Jul; July                               |
| d      | day-of-month                 | number(3)    | 28                                             |
| Q/q    | quarter-of-year              | number/text  | 3; 03; Q3; 3rd quarter                         |
| E      | day-of-week                  | text         | Tue; Tuesday                                   |
| F      | aligned day of week in month | number(1)    | 3                                              |
| a      | am-pm-of-day                 | am-pm        | PM                                             |
| h      | clock-hour-of-am-pm (1-12)   | number(2)    | 12                                             |
| K      | hour-of-am-pm (0-11)         | number(2)    | 0                                              |
| k      | clock-hour-of-day (1-24)     | number(2)    | 0                                              |
| H      | hour-of-day (0-23)           | number(2)    | 0                                              |
| m      | minute-of-hour               | number(2)    | 30                                             |
| s      | second-of-minute             | number(2)    | 55                                             |
| S      | fraction-of-second           | fraction     | 978                                            |
| V      | time-zone ID                 | zone-id      | America/Los_Angeles; Z; -08:30                 |
| z      | time-zone name               | zone-name    | Pacific Standard Time; PST                     |
| O      | localized zone-offset        | offset-O     | GMT+8; GMT+08:00; UTC-08:00;                   |
| X      | zone-offset ‘Z’ for zero     | offset-X     | Z; -08; -0830; -08:30; -083015; -08:30:15;     |
| x      | zone-offset                  | offset-x     | +0000; -08; -0830; -08:30; -083015; -08:30:15; |
| Z      | zone-offset                  | offset-Z     | +0000; -0800; -08:00;                          |
| ‘      | escape for text              | delimiter    |                                                |
| ’‘     | single quote                 | literal      | ’                                              |
| [      | optional section start       |              |                                                |
| ]      | optional section end         |              |                                                |

# Data Types


* [Data Types](https://spark.apache.org/docs/latest/sql-ref-datatypes.html#data-types)

```from pyspark.sql.types import *```

| Data type | Value type in Python | API to access or create a data type |  |
|:---|:---|:---|:--|
|ByteType | int or long Note: Numbers will be converted to 1-byte signed integer numbers at runtime. Please make sure that numbers are within the range of -128 to 127. | ByteType() |  |
| ShortType | int or long Note: Numbers will be converted to 2-byte signed integer numbers at runtime. Please make sure that numbers are within the range of -32768 to 32767. | ShortType() |  |
| IntegerType | int or long | IntegerType() |  |
| LongType | long Note: Numbers will be converted to 8-byte signed integer numbers at runtime. Please make sure that numbers are within the range of -9223372036854775808 to 9223372036854775807.Otherwise, please convert data to decimal.Decimal and use DecimalType. | LongType() |  |
| FloatType | float Note: Numbers will be converted to 4-byte single-precision floating point numbers at runtime. | FloatType() |  |
| DoubleType | float | DoubleType() |  |
| DecimalType | decimal.Decimal | DecimalType() |  |
| StringType | string | StringType() |  |
| BinaryType | bytearray | BinaryType() |  |
| BooleanType | bool | BooleanType() |  |
| TimestampType | datetime.datetime | TimestampType() |  |
| DateType | datetime.date | DateType() |  |
| ArrayType | list, tuple, or array | ArrayType(elementType, [containsNull]) Note:The default value of containsNull is True. |  |
| MapType | dict | MapType(keyType, valueType, [valueContainsNull]) Note:The default value of valueContainsNull is True. |  |
| StructType | list or tuple | StructType(fields) Note: fields is a Seq of StructFields. Also, two fields with the same name are not allowed. |  |
| StructField | The value type in Python of the data type of this field (For example, Int for a StructField with the data type IntegerType) | StructField(name, dataType, [nullable]) Note: The default value of nullable is True. |  |


# CSV

In [ ]:
%%bash
hdfs dfs -mkdir -p flight
hdfs dfs -put -f ./data/flight/*.csv flight/

In [ ]:
flight = spark.read\
    .option("compression", "none")\
    .option("header", True)\
    .option("sep", ',')\
    .option("nullValue", np.nan)\
    .option("inferSchema", True)\
    .option("dateFormat", "yyyy-MM-dd")\
    .csv("flight/flightData.csv")\
    .withColumn("date", to_date(col("date"), "yyyy-MM-dd"))

flight.printSchema()
flight.createOrReplaceTempView("flight")

In [ ]:
flight.show()

---
# Stop Spark Session

In [34]:
spark.stop()



# Cleanup

In [35]:
del spark
gc.collect()

1683